In [3]:
from llama_cpp import Llama

In [6]:
llm = Llama(model_path='/Users/kunalbhandarkar/Downloads/DeepSeek-R1-Distill-Qwen-1.5B-Q4_K_M.gguf')

llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 12287 MiB free
llama_model_loader: loaded meta data with 30 key-value pairs and 339 tensors from /Users/kunalbhandarkar/Downloads/DeepSeek-R1-Distill-Qwen-1.5B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = DeepSeek R1 Distill Qwen 1.5B
llama_model_loader: - kv   3:                           general.basename str              = DeepSeek-R1-Distill-Qwen
llama_model_loader: - kv   4:                         general.size_label str              = 1.5B
llama_model_loader: - kv   5:                          qwen2.block_count u32              = 28
llama_m

In [10]:
output = llm("tell me a joke in 10 words or less")
print(output)

Llama.generate: 1 prefix-match hit, remaining 11 prompt tokens to eval
llama_perf_context_print:        load time =    1218.24 ms
llama_perf_context_print: prompt eval time =     108.87 ms /    11 tokens (    9.90 ms per token,   101.04 tokens per second)
llama_perf_context_print:        eval time =     212.37 ms /    15 runs   (   14.16 ms per token,    70.63 tokens per second)
llama_perf_context_print:       total time =     327.65 ms /    26 tokens


{'id': 'cmpl-a8728c87-201b-45dc-a1ff-5c0640f678e9', 'object': 'text_completion', 'created': 1741582113, 'model': '/Users/kunalbhandarkar/Downloads/DeepSeek-R1-Distill-Qwen-1.5B-Q4_K_M.gguf', 'choices': [{'text': ' that involves the word "gossiping." Can\'t use any markdown, just', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 12, 'completion_tokens': 16, 'total_tokens': 28}}


In [11]:
print(output['choices'][0]['text'])

 that involves the word "gossiping." Can't use any markdown, just
